In [13]:
%matplotlib inline

import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt
import seaborn as sns

from pyproj import Proj, transform

from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import DBSCAN

import bokeh
import bokeh.plotting as plotting
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.tile_providers import CARTODBPOSITRON
plotting.output_notebook()

sns.set_style('whitegrid')

Loading BokehJS ...

# Problem definition

# Load the data

In [18]:
# TO DO #1: LOAD THE DATASET
df = pd.read_csv('/Users/jeromepotvin/Desktop/PERSONNEL JEROME/CONCORDIA/1260_BIG DATA/PROJECT/DATA/RAW/1_FINAL CUSTOMER SALES_16_17.csv', encoding='latin_1')
#df['Product_Family_Desc'] = pd.to_datetime(df['Product_Family_Desc'])
print(df.columns)
print((df['Market_Segment_Desc ']).value_counts())
print((df['Ship_Type']).value_counts())
df.head()

Index(['Customer_Name ', 'Product_Family_Desc', 'Ship_Type', 'Order Type',
       'Customer_Address ', 'Market_Segment_Desc ', 'Supplier_Name', '2016',
       '2017'],
      dtype='object')
COMMERCIAL CONTRACTOR      83041
INDUSTRY END USERS         15701
OTHER                      10123
DATA CONTRACTORS            9861
GOVERNMENT INSTITUTIONS     9752
INDUSTRIAL CONTRACTOR       5333
RESIDENTIAL CONTRACTOR      4678
FACILITY MANAGEMENT         3458
UTILITIES                   1563
UTILITY CONTRACTORS          572
DATA END USERS               386
Name: Market_Segment_Desc , dtype: int64
1    122399
2     17329
3      4740
Name: Ship_Type, dtype: int64


,Customer_Name,Product_Family_Desc,Ship_Type,Order Type,Customer_Address,Market_Segment_Desc,Supplier_Name,2016,2017
0,Customer_Name,WIRE & CABLE,1,A,"SAINT-LAURENT, H4T1X7",OTHER,SPOOLON MFG LTD,"$2,011.52","$3,017.28"
1,Customer_Name,CONDUIT & ACCESSORIES,1,C,"SAINT-LAURENT, H4T1X7",OTHER,COLUMBIA/MBF,$41.19,NaN
2,Customer_Name,CONDUIT & ACCESSORIES,1,C,"SAINT-LAURENT, H4T1X7",OTHER,REXEL CANADA ELECTRICAL INC.,$29.87,$4.35
3,Customer_Name,CONDUIT & ACCESSORIES,1,C,"SAINT-LAURENT, H4T1X7",OTHER,THOMAS & BETTS,NaN,$22.27
4,Customer_Name,"CONNECT, FITT & OUT/BOXES",1,A,"SAINT-LAURENT, H4T1X7",OTHER,3M CDA INC,$13.00,NaN


In [ ]:
# Printed Ship type, in order to evaluate which one could represent the highset value from our warehouse sales
#Ship type 1: warehouse sales, Ship type 2: Nt listed items Ship type 3: shipment from manufacturer

# Feature Engineering 

In [28]:
# feature engineering

# TO DO #2: SELECT A Product family
#df = df[(df['Product_Family_Desc'])>'10000')]
df = df[(df['Product_Family_Desc'])>'10000']

# TO DO #3: SELECT A CATEGORY
#df = df[df['Market_Segment_Desc ']=='COMMERCIAL CONTRACTOR']

# TO DO #4: REMOVE SELECT LINES WITH NO LOCATION
#df = df[(['X']>0)&(df['Y'])>0]

# adapt X and Y to the visualization
df['X'] = df.apply(lambda x: transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), x['LONG'], x['LAT'])[1], axis=1)
df['Y'] = df.apply(lambda x: transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), x['LONG'], x['LAT'])[0], axis=1)

#X_columns = ['X', 'Y']
df = df[X_columns]
print(df.shape)


KeyError: ('LONG', 'occurred at index 0')

In [25]:
#investigate with Commercial contractor segment since it represents mors than 50% of all transactions

# Model Training

In [42]:
# TO DO #5: INSTANTIATE THE DBSCAN MODEL WITH THE PARAMETERS eps AND min_samples
model = DBSCAN(eps=1000, min_samples=5) 

# TO DO #6: FIT THE COLUMNS 'X' AND 'Y'
model.fit(df[['X', 'Y']])

cluster_labels = model.labels_
n_clusters = len(set(cluster_labels))
# TO DO #6: THE SIZE OF EACH CLUSTER
print(collections.Counter(cluster_labels))

df['cluster'] = cluster_labels

Counter({0: 90281, 1: 2738, 2: 26, 4: 18, 5: 6, 3: 5, -1: 4})


In [5]:
p = figure(y_range=(5641788.0, 5751788.0), x_range=(-8152883, -8252883))
p.add_tile(CARTODBPOSITRON)

latitude  = list(df[df['cluster']>-1]['X'].values)
longitude = list(df[df['cluster']>-1]['Y'].values)

colormap = list(bokeh.palettes.viridis(n_clusters))
colors = [colormap[x] for x in df[df['cluster']>-1]['cluster']]
source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude))
p.circle(x=longitude, y=latitude, color=colors, fill_alpha=0.2, size=5)
show(p)

# Model Evaluation

In [6]:
# TO DO #7: CALCULATE 'Intra-Cluster Distances' AND 'Inter-Cluster Distances'


('Intra-Cluster Distances', 2268302.4871340226)
('Inter-Cluster Distances', 1924666.6823221939)
